# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Air Quality** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [9]:
import sys
from pathlib import Path
import re
def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/zohra/mlfs-book
HopsworksSettings initialized!


In [10]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs.airquality import util
import os
import importlib
importlib.reload(util)

<module 'mlfs.airquality.util' from '/home/zohra/mlfs-book/mlfs/airquality/util.py'>

In [11]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

datetime.datetime(2025, 11, 17, 16, 23, 22, 389180)

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [12]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

2025-11-17 16:23:22,427 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-17 16:23:22,457 INFO: Initializing external client
2025-11-17 16:23:22,458 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-17 16:23:23,550 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-11-17 16:23:24,376 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279126


## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [13]:
mr = project.get_model_registry()
air_quality_fg = fs.get_feature_group(
    name='air_quality_per_city',
    version=2,
)
target_city = "catalunya"

city_info = next(
    loc for loc in location 
    if loc["city"].lower() == target_city.lower()
)
country = city_info["country"]
city =target_city
weather_fg = fs.get_feature_group(
    name='weather_spain',
    version=1,
)
selected_features = (
    air_quality_fg
    .select(['pm25', 'date', 'rolling_mean', 'city', 'street'])
    .join(
        weather_fg.select_features(),
        on=['city']
    )
)

selected_features_df = selected_features.read().sort_values("date")


streets = selected_features_df["street"].dropna().unique()
models_by_street = {}

for s in streets:
    safe_street = re.sub(r"[^A-Za-z0-9]+", "_", s)
    model_name = "air_quality_xgboost_" + safe_street

    try:
        # 1) Get all versions of this model name
        all_versions = mr.get_models(name=model_name)

        if not all_versions:
            print(f" No model versions found for street '{s}' with name '{model_name}'")
            continue

        # 2) Take the latest by version number
        latest_model = max(all_versions, key=lambda m: m.version)

        models_by_street[s] = latest_model
        print(f" Found model '{model_name}' v{latest_model.version} for street '{s}'")

    except Exception as e:
        print(f" Error fetching model for street '{s}' with name '{model_name}': {e}")


2025-11-17 16:23:27,248 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather_spain` as features for the query. To include primary key and event time use `select_all`.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.18s) 
 Found model 'air_quality_xgboost_barcelona_gracia_st_gervasi' v13 for street 'barcelona-gracia-st.gervasi'
 Found model 'air_quality_xgboost_barcelona_parc_vall_dhebron' v13 for street 'barcelona-parc-vall-dhebron'
 Found model 'air_quality_xgboost_barcelona' v13 for street 'barcelona'
 Found model 'air_quality_xgboost_barcelona_eixample' v12 for street 'barcelona-eixample'
 Found model 'air_quality_xgboost_barcelona_palau_reial' v10 for street 'barcelona-palau-reial'
 Found model 'air_quality_xgboost_barcelona_poblenou' v10 for street 'barcelona-poblenou'


In [14]:
import os
from xgboost import XGBRegressor

xgb_models = {} 

for street, model_meta in models_by_street.items():
    print(street, "->", model_meta)
    # 1) Download the directory for this street’s model
    saved_model_dir = model_meta.download()
    retrieved_xgboost_model = XGBRegressor()
    safe_street = re.sub(r"[^A-Za-z0-9]+", "_", street) 
    retrieved_xgboost_model.load_model(saved_model_dir + f"/model_{safe_street}.json")
    xgb_models[street] = retrieved_xgboost_model
    print(f" Loaded XGBoost model for {street}")
"""
    # 2) Find the model JSON file automatically
    model_json = None
    for f in os.listdir(model_dir):
        if f.endswith(".json"):
            model_json = os.path.join(model_dir, f)
            break

    if model_json is None:
        print(f" No JSON model file found for {street}")
        continue

    # 3) Load into XGBoost
    model = XGBRegressor()
    model.load_model(model_json)
"""
    


barcelona-gracia-st.gervasi -> Model(name: 'air_quality_xgboost_barcelona_gracia_st_gervasi', version: 13)


Downloading: 0.000%|          | 0/443483 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/584308 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/461175 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/459322 elapsed<00:00 remaining<?

...

Downloading: 0.000%|          | 0/28078 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/28515 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/18686 elapsed<00:00 remaining<?

 Loaded XGBoost model for barcelona-poblenou)... DONE


'\n    # 2) Find the model JSON file automatically\n    model_json = None\n    for f in os.listdir(model_dir):\n        if f.endswith(".json"):\n            model_json = os.path.join(model_dir, f)\n            break\n\n    if model_json is None:\n        print(f" No JSON model file found for {street}")\n        continue\n\n    # 3) Load into XGBoost\n    model = XGBRegressor()\n    model.load_model(model_json)\n'

In [15]:
air_quality_df = air_quality_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.49s) 


In [16]:
batch_data = weather_fg.filter(weather_fg.date >= today).read()
m=batch_data.sort_values("date", ascending=True).copy()
m

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
1,2025-11-18 00:00:00+00:00,6.45,0.0,10.464797,243.435013,catalunya
4,2025-11-19 00:00:00+00:00,4.55,0.0,9.227524,249.443878,catalunya
0,2025-11-20 00:00:00+00:00,6.25,0.0,12.181625,251.029495,catalunya
2,2025-11-21 00:00:00+00:00,2.70,0.0,20.018072,307.694305,catalunya
3,2025-11-22 00:00:00+00:00,6.30,0.0,22.668568,280.060638,catalunya
5,2025-11-23 00:00:00+00:00,9.00,0.0,14.934577,254.623688,catalunya


In [17]:
def recursive_predict_with_rolling_3(model, air_quality_df, batch_data):
   

    # Make sure historical data is ordered from most recent to oldest
    hist = air_quality_df.sort_values("date", ascending=False).copy()

    # last 3 *real* pm25 values (most recent first)
    history = list(hist["pm25"].iloc[:3])

    batch_sorted = batch_data.sort_values("date", ascending=True).copy()

    rolling_means = []

    feature_cols = [
        "rolling_mean",
        "temperature_2m_mean",
        "precipitation_sum",
        "wind_speed_10m_max",
        "wind_direction_10m_dominant",
    ]

    for _, row in batch_sorted.iterrows():
    
        window = history[:3]
        rolling_mean = sum(window) / len(window)
        rolling_means.append(rolling_mean)

        # --- build features for this horizon ---
        row_features = row.copy()
        row_features["rolling_mean"] = rolling_mean
        
        X = row_features[feature_cols].to_frame().T
        X = X.astype(float)

        y_hat = model.predict(X)[0]

        history = [y_hat] + history[:2]

    batch_sorted["rolling_mean"] = rolling_means

    # put back in original order if needed
    return batch_sorted


In [18]:
batch_data_by_street = {}
for s in xgb_models.keys():
    batch_data_by_street[s] = recursive_predict_with_rolling_3(
    model=xgb_models[s],
    air_quality_df = air_quality_df[air_quality_df["street"] == s],
    batch_data=batch_data,
)

In [19]:
debug=pd.to_datetime(batch_data_by_street['barcelona']['date']).dt.date
debug

1    2025-11-18
4    2025-11-19
0    2025-11-20
2    2025-11-21
3    2025-11-22
5    2025-11-23
Name: date, dtype: object

In [20]:
for s in batch_data_by_street.keys():
    batch_data_by_street[s]['predicted_pm25'] = xgb_models[s].predict(
    batch_data_by_street[s][['rolling_mean','temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']])


In [21]:
for s in batch_data_by_street.keys():
    batch_data_by_street[s]['street'] = s
    batch_data_by_street[s]['city'] = city
    batch_data_by_street[s]['country'] = country
    # Fill in the number of days before the date on which you made the forecast (base_date)
    batch_data_by_street[s]['days_before_forecast_day'] = range(1, len(batch_data)+1)
    batch_data_by_street[s] = batch_data_by_street[s].sort_values(by=['date'])

In [ ]:
for street, data in batch_data_by_street.items():
    path_name = re.sub(r"[^A-Za-z0-9]+","_", street)
    pred_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_forecast_{path_name}.png"
    plt = util.plot_air_quality_forecast(city, street, data, pred_file_path)
    plt.show()

2025-11-17 16:25:35,758 WARNING: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.



In [ ]:
batch_data_by_street['barcelona'].info()

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [ ]:
# Get or create feature group
monitors_fg_by_street = {}
monitors_df_by_street = {}
for street in streets:
    feature_name = 'aq_predictions_'+ re.sub(r"[^A-Za-z0-9]+","_", street)
    monitor_fg = fs.get_or_create_feature_group(
        name=feature_name,
        description='Air Quality prediction monitoring',
        version=2,
        primary_key=['city','street','date','days_before_forecast_day'],
        event_time="date"
    )
    monitor_fg.insert(batch_data_by_street[street], wait=True)
    monitors_df_by_street[street] = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
    monitors_fg_by_street[street] = monitor_fg

In [ ]:
monitors_fg_by_street['barcelona'].read().info()

In [ ]:
air_quality_fg = fs.get_feature_group(name='air_quality_per_city', version=2)
air_quality_df = air_quality_fg.read()
air_quality_df

In [ ]:
hindcasts={}
for street, monitor in monitors_df_by_street.items():
    outcome_df = air_quality_df[air_quality_df["street"] == street][['date', 'pm25']]
    preds_df =  monitor[['date', 'predicted_pm25']]
    
    hindcast_df = pd.merge(preds_df, outcome_df, on="date")
    hindcast_df = hindcast_df.sort_values(by=['date'])
    
    # If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
    if len(hindcast_df) == 0:
        hindcast_df = util.backfill_predictions_for_monitoring_per_street(weather_fg, air_quality_fg, monitors_fg_by_street[street], xgb_models[street],street)
    hindcasts[street]=hindcast_df

### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

__This graph will be empty to begin with - this is normal.__

After a few days of predictions and observations, you will get data points in this graph.

In [ ]:
for s, hindcast in hindcasts.items():
    path_name = re.sub(r"[^A-Za-z0-9]+","_", s) 
    hindcast_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_hindcast_1day_{path_name}.png"
    plt = util.plot_air_quality_forecast(city, s, hindcast, hindcast_file_path, hindcast=True)
    plt.show()

### Upload the prediction and hindcast dashboards (png files) to Hopsworks


In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/airquality") == False:
    dataset_api.mkdir("Resources/airquality")
for s in streets:
    dataset_api.upload(pred_file_path, f"Resources/airquality/{city}_{street}_{str_today}_{path_name}", overwrite=True)
    dataset_api.upload(hindcast_file_path, f"Resources/airquality/{city}_{street}_{str_today}_{path_name}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/airquality_v2")

---